In [526]:
import numpy as np
from numpy.linalg import multi_dot
from numpy.linalg import inv


In [527]:
def kalman_1d(belief, sigma_v, sigma_w, u, z):
    x = belief[0]
    var = belief[1]

    # prediction
    x_pred = x + u
    var_pred = var + sigma_v

    # innovation
    nu = z - x_pred
    k = var_pred/(var_pred+sigma_w)

    # update
    x_new = x_pred + k*nu
    var_new = (1 - k) * var_pred

    posterior = [x_new, var_new]
    print(np.around(posterior, 5))
    print()

    return posterior


In [528]:
sigma_v = 0.04
sigma_w = 0.01
U = [-0.5, 1.2, 0.3]
Z = [-0.7, 0.6, 0.95]
belief = [0, 1]

for i in range(3):
    belief = kalman_1d(belief, sigma_v, sigma_w, U[i], Z[i])


[-0.6981  0.0099]

[0.58362 0.00833]

[0.93862 0.00829]



In [529]:
def kalman_nd(belief, Sigma_v, Sigma_w, u, z):
    [r, c] = np.shape(belief)
    x = np.array(belief[:, 0])
    var = belief[:, 1:]

    # prediction
    x_pred = x + u
    var_pred = var + Sigma_v

    # innovation
    nu = z - x_pred
    K = var_pred*np.linalg.inv(var_pred+Sigma_w)

    # update
    x_new = x_pred + np.matmul(K, nu)
    var_new = np.matmul((np.identity(r) - K), var_pred)

    posterior = np.vstack((x_new, var_new))
    print(np.around(posterior.T, 5))
    print()

    return posterior.T


In [530]:
sigma_v = np.array([[0.04, 0],
                    [0, 0.09]])

sigma_w = np.array([[0.01, 0],
                    [0, 0.02]])

U = np.array([[-0.5, 1.2, 0.3],
              [0.3, -0.6, 0.3]])

Z = np.array([[-0.7, 0.6, 0.95],
              [0.3, 0.0, 0.15]])

belief = np.array([[0, 1, 0],
                   [0, 0, 1]])

for i in range(3):
    belief = kalman_nd(belief, sigma_v, sigma_w, U[:, i], Z[:, i])


[[-0.6981   0.0099   0.     ]
 [ 0.3      0.       0.01964]]

[[ 0.58362  0.00833  0.     ]
 [-0.04628  0.       0.01691]]

[[0.93862 0.00829 0.     ]
 [0.16634 0.      0.01685]]



# 1-d Kalman Filter with differential motion

$$
Noise =
\begin{bmatrix}
0.02 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0.03
\end{bmatrix}
$$


- Prediction Step
  $$
  X_{t+1}^+ = FX_t + Gu_t
  $$

$$
\begin{bmatrix}
x \\
\dot{x} \\
\ddot{x}
\end{bmatrix}_{t+1}^+
=
\begin{bmatrix}
1 & dt & 0\\
0 & 1 & 0\\
0 & 0 & 0
\end{bmatrix}

\begin{bmatrix}
x \\
\dot{x} \\
\ddot{x}
\end{bmatrix}_t
+
\frac{1}{m}
\begin{bmatrix}
\frac{1}{2}dt^2 \\
dt \\
1
\end{bmatrix}
u_t
$$

$$
P_{t+1}^+ = FP_tF^T+V
$$

$$
\begin{bmatrix}
\sigma_{pos} & 0 & 0\\
0 & \sigma_{vel} & 0\\
0 & 0 & \sigma_{acc}
\end{bmatrix}_{t+1}^+
=
\begin{bmatrix}
1 & dt & 0\\
0 & 1 & 0\\
0 & 0 & 0
\end{bmatrix}

\begin{bmatrix}
\sigma_{pos} & 0 & 0\\
0 & \sigma_{vel} & 0\\
0 & 0 & \sigma_{acc}
\end{bmatrix}_{t}

\begin{bmatrix}
1 & dt & 0\\
0 & 1 & 0\\
0 & 0 & 0
\end{bmatrix}^T
+
Noise
$$

- Innovation Step
  $$
  \nu = z_{t+1} - HX_{t+1}^+
  $$

$$
\nu = z_{t+1} -
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix}

\begin{bmatrix}
x \\
\dot{x} \\
\ddot{x}
\end{bmatrix}_{t+1}^+
$$


* Update Step
$$
X_{t+1} = X_{t+1}^+ + K\nu
$$

$$
P_{t+1} = (I - KH)P_{t+1}^+
$$

In [531]:
def kalman_1dmotion(belief, Sigma_v, Sigma_w, u, z, F, G, H):
    [r, c] = np.shape(belief)
    x = np.array(belief[:, 0])
    x = np.reshape(x, [r, 1])
    var = belief[:, 1:]

    # prediction
    Gu = np.dot(G, u)
    x_pred = np.matmul(F, x) + Gu
    var_pred = multi_dot([F, var, F.T]) + Sigma_v

    # innovation
    nu = z - np.matmul(H, x_pred)
    HPHW = multi_dot([H, var_pred, H.T]) + Sigma_w
    K = multi_dot([var_pred, H.T, inv(HPHW)])

    # update
    x_new = x_pred + K*nu
    I = np.identity(r)
    KH = np.matmul(K, H)
    var_new = np.matmul((I - KH), var_pred)

    posterior = np.hstack((x_new, var_new))

    return posterior


In [532]:
sigma_v = np.array([[0.02, 0, 0],
                    [0, 0.03, 0],
                    [0, 0, 0.]])

sigma_w = 0.01

U = np.array([-0.2, 0.0, 0.1])

Z = np.array([0.4, 0.9, 0.8])

belief = np.array([[0, 1, 0, 0],
                   [0.5, 0, 2, 0],
                   [0, 0, 0, 0]])

dt = 1

F = np.array([[1, dt, 0],
             [0, 1, 0],
             [0, 0, 0]])

m = 1
G = 1/m * np.array([[0.5*dt**2], [dt], [1]])

H = np.array([1, 0, 0])
H = np.reshape(H, [1, 3])

for i in range(3):
    print(belief)
    print()
    belief = kalman_1dmotion(belief, sigma_v, sigma_w, U[i], Z[i], F, G, H)
print(belief)


[[0.  1.  0.  0. ]
 [0.5 0.  2.  0. ]
 [0.  0.  0.  0. ]]

[[ 0.4         0.009967    0.00660066  0.        ]
 [ 0.3         0.00660066  0.70986799  0.        ]
 [-0.2         0.          0.          0.        ]]

[[0.89737889 0.00986894 0.00938971 0.        ]
 [0.48779412 0.00938971 0.067125   0.        ]
 [0.         0.         0.         0.        ]]

[[0.8505014  0.00920492 0.00608354 0.        ]
 [0.20138417 0.00608354 0.05057698 0.        ]
 [0.1        0.         0.         0.        ]]
